In [2]:
import json

In [ ]:
# !wget http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_distractor_v1.json
# !wget http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_fullwiki_v1.json
# !wget http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json

# !mkdir hotpot_qa
# !mv hotpot_dev_distractor_v1.json hotpot_qa/
# !mv hotpot_dev_fullwiki_v1.json hotpot_qa/
# !mv hotpot_train_v1.1.json hotpot_qa/

In [3]:
path = "hotpot_qa/hotpot_train_v1.1.json"
with open(path) as f:
    data = json.load(f)

In [4]:
data[0]

{'supporting_facts': [["Arthur's Magazine", 0], ['First for Women', 0]],
 'level': 'medium',
 'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'context': [['Radio City (Indian radio station)',
   ["Radio City is India's first private FM radio station and was started on 3 July 2001.",
    ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
    ' It plays Hindi, English and regional songs.',
    ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.',
    ' Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.',
    ' The Radio station currently plays a mix of Hindi and Regional music.',
    ' Abraham Thomas is the CEO of the company.']]

In [5]:
import pandas as pd # type: ignore

In [6]:
mp = {}
for i in range(len(data)):
    d = data[i]
    for x,y in d['context']:
        mp[x] = ''.join(y)

In [7]:
doc_df = pd.DataFrame(list(mp.items()), columns=['heading', 'context'])
doc_df.head()

,heading,context
0,Radio City (Indian radio station),Radio City is India's first private FM radio s...
1,History of Albanian football,Football in Albania existed before the Albania...
2,Echosmith,"Echosmith is an American, Corporate indie pop ..."
3,Women's colleges in the Southern United States,Women's colleges in the Southern United States...
4,First Arthur County Courthouse and Jail,"The First Arthur County Courthouse and Jail, w..."


In [8]:
doc_df.to_csv('train_docs.csv', index=False)
len(doc_df)

482021

In [9]:
dataset = []
for i in range(len(data)):
    d = data[i]
    mp = {x:''.join(y) for x,y in d['context']}
    for l1 in d['supporting_facts']:
        pos_doc = mp[l1[0]]
        dataset.append({
            'query':d['question'], 
            'pos_doc':pos_doc
            })

In [10]:
(dataset)[:2]

[{'query': "Which magazine was started first Arthur's Magazine or First for Women?",
  'pos_doc': 'Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. In May 1846 it was merged into "Godey\'s Lady\'s Book".'},
 {'query': "Which magazine was started first Arthur's Magazine or First for Women?",
  'pos_doc': "First for Women is a woman's magazine published by Bauer Media Group in the USA. The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies."}]

In [11]:
df = pd.read_csv('train_docs.csv')
all_docs = list(df.context)
len(all_docs)

482021

In [12]:
type(all_docs)

list

In [13]:
final_dict = []
N = 4
for i in range(len(dataset)):
    query = dataset[i]['query']
    pos_doc = dataset[i]['pos_doc']
    import random
    random.seed(100*(i+1))
    neg_docs = random.sample(all_docs, N)
    final_dict.append({
        'query':query,
        'positive_document': pos_doc,
        'negative_documents': neg_docs
    })

In [14]:
final_dict[0]

{'query': "Which magazine was started first Arthur's Magazine or First for Women?",
 'positive_document': 'Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. In May 1846 it was merged into "Godey\'s Lady\'s Book".',
 'negative_documents': ['Richard Burbage (6 January 1567 – 12 March 1619) is considered the first great actor of English theatre. He is one of the most famous actors of the Globe Theatre of his time. In addition to being a stage actor he was also a theatre owner, entrepreneur and painter. He was the younger brother of Cuthbert Burbage. They were both actors in drama. Burbage was a business associate and friend to William Shakespeare.',
  'Jack Clayton (1 March 1921 – 26 February 1995) was a British film director and producer, who specialised in bringing literary works to the screen.',
  'Plaza

In [15]:
len(final_dict)

215684

In [16]:
with open('train_data.json', 'w') as f:
    json.dump(final_dict, f, indent=4)

In [18]:
from datasets import Dataset # type: ignore

dataset = Dataset.from_list(final_dict)

In [19]:
len(dataset)

215684